In [11]:
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)


# Okay, but seriously there's a big problem

The LSTM is trained on min-max scaled data from 2024-2025.

But when we do the RL, the min-max scaling happens from 2025-present.

So the min-max scaling isn't even the same.

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Generate synthetic data for demonstration
# Features: 1000 samples, each with 10 timesteps and 1 feature
# Targets: 1000 regression values
np.random.seed(42)
X = np.random.rand(1000, 10, 1)
y = np.random.rand(1000)

# Define the LSTM regression model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(10, 1)),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=20, batch_size=32, verbose=1)

# Example prediction
sample_input = np.random.rand(1, 10, 1)  # Single sample with 10 timesteps
prediction = model.predict(sample_input)
print("Prediction:", prediction)

Epoch 1/20


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2675  
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0871
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0847
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0847 
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0861 
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0844 
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0822 
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0835 
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0868
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0827 
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0821 
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0857 
Epoch 13/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0829
Epoch 14/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0846 
Epoch 15/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0809 
Epoch 16/20
32/32 

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# Define the number of splits for k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

best_model = None
best_r2 = float('-inf')

# Perform k-fold cross-validation
for train_index, val_index in kf.split(X):
    # Split the data into training and validation sets
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Define a new model for each fold
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(10, 1)),
        Dense(1)  # Output layer for regression
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    
    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    
    # Predict on the validation set
    y_pred = model.predict(X_val)
    
    # Calculate R² score
    r2 = r2_score(y_val, y_pred)
    print(f"Fold R²: {r2}")
    
    # Update the best model if the current one is better
    if r2 > best_r2:
        best_r2 = r2
        best_model = model

print(f"Best R²: {best_r2}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Fold R²: -0.031944932970606965


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold R²: -0.008463300280357355


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Fold R²: -0.011218091656060247


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold R²: -0.0046443238333682135


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold R²: 0.00782404296364747
Best R²: 0.00782404296364747


In [7]:
def a(x):
    a.b = 5
    def updateB():
        a.b = 10

    updateB()

    return x + a.b
a(5)

15

In [84]:
import pandas as pd
import optuna

TICKER = "ETHUSDC"
STOP_DATE = pd.Timestamp(year=2025, month=1, day=1, hour=12)
LONGEST_PERIOD_MONTHS = 12
NUM_FOLDS = 4 # note: this will test NUM_FOLDS-1 times

# optuna hyperparams
def HYPERPARAMS(funct, **kwargs):
    def objective(trial):
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
        batch_size = trial.suggest_int("batch_size", 16, 128)
        epochs = trial.suggest_int("epochs", 10, 50)
        activation = trial.suggest_categorical("activation", ["relu", "tanh"])
        lstm_units = trial.suggest_int("lstm_units", 32, 128)
        
        hyperparams = {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'epochs': epochs,
            'lstm_units': lstm_units,
            'activation': activation,
        }
        return funct(hyperparams=hyperparams, **kwargs)
    return objective

In [85]:
"""
=== Q&A ===

RL Model
[      Train      ] [  Test  ]
[  Train  ]  [Test]
LSTM Model


Q: Will this program only have access to "train" data? Should it have access to "test" data?
A: It should not have access to "test" data. If we want to see how it performs on test data,
we run a separate experiment from "training". Even automated performance monitoring on the
"test" data will be conducted globally, not locally.

Q: How will this program optimize the LSTM model before handing it off?
Step 1: generate Walk-Forward Test Folds (A test B, AB test C, ABC test D, ... A-Y test Z)
Step 2: use optuna to find the hyperparameters that have the best average MSE
Step 3: train those hyperparameters on A-Z, publish

Q: What financial indicators? What target?
A: No financial indicators
   Predicts "Open"

"""


from sklearn import preprocessing
def getAssetData(ticker, maxPeriodLength_months=12):
    df = pd.read_csv(f'assets/{ticker}.csv')

    # filter to longest contiguous before STOP_DATE
    df.drop(columns=["Ignore", "Close time"], inplace=True)
    df["Open time"] = df["Open time"].apply(lambda x: x * 1000 if len(str(x)) < len("1738369800000000") else x)
    df["Open time"] = pd.to_datetime(df["Open time"], unit='us')
    df.set_index("Open time", inplace=True)
    df.sort_index(inplace=True)

    df = df[df.index < STOP_DATE]

    df["Gap"] = df.index.to_series().diff().dt.total_seconds().div(60).fillna(0)
    valid_intervals = df[df["Gap"] <= 30]
    last_gap_index = df[df["Gap"] > 30].index[-1]
    df = df.loc[last_gap_index:]
    

    df.drop(columns=["Gap"], inplace=True)

    # add financial indicators

    # chop off N/A or longer than maxPeriodLength
    df.dropna(inplace=True)
    df = df[df.index > (df.index[-1] - pd.DateOffset(months=maxPeriodLength_months))]

    # normalize except the target
    target = df["Open"].copy()
    scaler = preprocessing.MinMaxScaler()
    normalized_columns = scaler.fit_transform(df)
    df = pd.DataFrame(normalized_columns, columns=df.columns, index=df.index)
    df["Target"] = target

    return df

def formatLSTMInput(df, numFolds=4, lookback=5):
    foldEndIndices = [int(len(df) * (i + 1) / numFolds) for i in range(numFolds)]
    foldEndIndices = [0] + foldEndIndices + [len(df)]

    folds = [df.iloc[foldEndIndices[i]:foldEndIndices[i + 1]] for i in range(len(foldEndIndices) - 1)]
    folds = [fold.reset_index(drop=True) for fold in folds]

    def formatXy(fold):
        if len(fold) <= lookback:
            print("Your fold is smaller than the lookback+1 period, so bro, fix it, please")
            return None, None
        X = pd.concat([fold.shift(i) for i in range(lookback, 0, -1)], axis=1).dropna()
        X = np.reshape(X.values, (len(X), lookback, len(fold.columns)))
        y = np.reshape(fold["Target"].shift(-lookback).dropna(), (-1, 1))
        return X, y

    # format X and format Y
    trainTestFolds = []
    for i in range(len(folds)-2):
        train_fold = pd.concat(folds[0:i + 1], axis=0)
        test_fold = folds[i + 1]
        X_train, y_train = formatXy(train_fold)
        X_test, y_test = formatXy(test_fold)
        trainTestFolds.append({
            'X_train': X_train,
            'y_train': y_train,
            'X_test': X_test,
            'y_test': y_test
        })
    
    train_final = pd.concat(folds, axis=0).reset_index(drop=True)
    X_final, y_final = formatXy(train_final)
    finalData = {
        'X_train': X_final,
        'y_train': y_final,
    }

    return { 'train-test-folds': trainTestFolds, 'final-train': finalData }

def generateModel(hyperparams):
    # build the Keras sequential model
    learning_rate = hyperparams.get('learning_rate', 0.001)
    batch_size = hyperparams.get('batch_size', 32)
    epochs = hyperparams.get('epochs', 20)
    lstm_units = hyperparams.get('lstm_units', 50)
    activation = hyperparams.get('activation', 'relu')
    input_shape = hyperparams.get('input_shape', (10, 1))

    model = Sequential([
        LSTM(lstm_units, activation='relu', input_shape=input_shape),
        Dense(1)  # Output layer for regression
    ])
    return model

def KFoldsTrainTest(hyperparams, trainTestFolds):
    hyperparams = hyperparams.copy()
    hyperparams['input_shape'] = trainTestFolds[0]['X_train'].shape[1:]

    # train the model on each fold and evaluate
    mse_scores = []
    for fold in trainTestFolds:
        model = generateModel(hyperparams)
        model.compile(optimizer='adam', loss='mse')
        X_train, y_train = fold['X_train'], fold['y_train']
        X_test, y_test = fold['X_test'], fold['y_test']

        # fit the model
        model.fit(X_train, y_train, epochs=hyperparams['epochs'], batch_size=hyperparams['batch_size'], verbose=0)

        # evaluate the model
        mse = model.evaluate(X_test, y_test, verbose=0)
        mse_scores.append(mse)

    # return the average MSE
    return np.mean(mse_scores)

def getOptimalModel(saveAs):
    data = getAssetData(TICKER, LONGEST_PERIOD_MONTHS)
    inputs = formatLSTMInput(data, NUM_FOLDS)
    
    trainTestfolds = inputs['train-test-folds']
    finalData = inputs['final-train']


    objective = HYPERPARAMS(KFoldsTrainTest, trainTestFolds=trainTestfolds)
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50)
    bestHyperparams = study.best_trial.params

    model = generateModel(bestHyperparams)
    model.compile(optimizer='adam', loss='mse')
    model.fit(finalData['X_train'], finalData['y_train'], epochs=bestHyperparams['epochs'], batch_size=bestHyperparams['batch_size'], verbose=1)
    model.save(saveAs)
    print(f"Best hyperparameters: {bestHyperparams}")

    return { 'model': model, 'hyperparams': bestHyperparams }

In [ ]:
getOptimalModel("test-regression.h5")

[I 2025-03-28 11:58:49,684] A new study created in memory with name: no-name-58aab716-9242-41d8-a551-83b6902b654f
c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2025-03-28 11:59:21,071] Trial 0 finished with value: 609.3888956705729 and parameters: {'learning_rate': 0.000889053279249898, 'batch_size': 68, 'epochs': 37, 'activation': 'tanh', 'lstm_units': 95}. Best is trial 0 with value: 609.3888956705729.
[I 2025-03-28 11:59:52,317] Trial 1 finished with value: 564.2031758626302 and parameters: {'learning_rate': 0.008266349674529452, 'batch_size': 124, 'epochs': 45, 'activation': 'relu', 'lstm_units': 108}. Best is trial 1 with value: 564.2031758626302.
[I 2025-03-28 12:00:14,630] Trial 2 finished with value: 604.9088745117188 an